<a href="https://colab.research.google.com/github/TakashiNinjouji/sbom-playground/blob/main/sbom_signing_wtih_Cosign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SBOM Signing with Cosign

## System check

Check the environment in Google Colaboratory.   
Linux: Ubuntu 22.04.2 LTS   


In [ ]:
!lsb_release -a

In [ ]:
!uname -a

In [ ]:
!apt -qq  update &&  apt -y upgrade && apt autoremove

## Go installation
Almost according to: "Download and install" `(https://go.dev/doc/install)  
go version go1.20.7 linux/amd64

In [ ]:
%%bash
wget https://go.dev/dl/go1.20.7.linux-amd64.tar.gz
tar -zxvf go1.20.7.linux-amd64.tar.gz
mv ./go /usr/local/
rm -rf go1.20.7.linux-amd64.tar.gz
mkdir /home/go

In [ ]:
import os
%env PATH={os.environ['PATH']}:/usr/local/go/bin
%env GOBIN=/usr/local/go/bin
%env GOPATH=/home/go

In [ ]:
!go version

## Cosign installation
https://github.com/sigstore/cosign  

In [ ]:
#### You can install the cosign binary
#### Current version: v2.1.1
%%bash
curl -O -L "https://github.com/sigstore/cosign/releases/latest/download/cosign-linux-amd64"
mv cosign-linux-amd64 /usr/local/go/bin/cosign
chmod +x /usr/local/go/bin/cosign

In [ ]:
!cosign version

## Signing
Almost according to the article:  
"A Step-by-Step Guide to Signing an SPDX SBOM with Sigstore’s Cosign"  
https://spdx.dev/a-step-by-step-guide-to-signing-an-spdx-sbom-with-sigstores-cosign/

In [ ]:
!cosign generate-key-pair

Prepare the sbom file which you will sign.  
In this notebook, we can use a spdx-json from "SPDX Usage Examples" by SPDX Workgroup:
https://github.com/spdx/spdx-examples

In [ ]:
!wget https://raw.githubusercontent.com/spdx/spdx-examples/master/presentations/OSS-NA-2023/SPDXVersion2.3/01-MinimalSBOM.json

In [ ]:
!cosign sign-blob --key cosign.key 01-MinimalSBOM.json --tlog-upload=false

The signature \<sig> will show as the result of the above command.  
Then, try to verify with the \<sig> using the following command:

In [ ]:
!cosign verify-blob --key cosign.pub --signature <sig> ./01-MinimalSBOM.json --insecure-ignore-tlog=true --verbose

---

# Notes


## Note 1. Cosign intalltion methods
1. Install directly from the repository  
>go install github.com/sigstore/cosign/v2/cmd/cosign@latest

2. Build from source
```
%%bash
git clone https://github.com/sigstore/cosign.git
cd cosign
make
mv ./cosign /usr/local/go/bin/cosign
```

3. NOT USE: v1.x will be installed...
> go install github.com/sigstore/cosign/cmd/cosign@latest



## Note 2. Command option to sign  
- For container: "cosign sign"   
- For file or blob: "cosign sign-blog"

### Note 3

You can sign using using the following command:   

> cosign sign-blob \--key \<path_to_private_key> \<spdx_sbom_file>


However, this command attempts to upload the artifact to the sigstore service hosted by LF Projects, LLC.

So, consider using the "--tlog-upload=false" option if you want to try it locally.


```
Using payload from: <spdx_sbom_file>
Enter password for private key:

	The sigstore service, hosted by sigstore a Series of LF Projects, LLC, is provided pursuant to the Hosted Project Tools Terms of Use, available at https://lfprojects.org/policies/hosted-project-tools-terms-of-use/.
	Note that if your submission includes personal data associated with this signed artifact, it will be part of an immutable record.
	This may include the email address associated with the account with which you authenticate your contractual Agreement.
	This information will be used for signing this artifact and will be stored in public transparency logs and cannot be removed later, and is subject to the Immutable Record notice at https://lfprojects.org/policies/hosted-project-tools-immutable-records/.

By typing 'y', you attest that (1) you are not submitting the personal data of any other person; and (2) you understand and agree to the statement and the Agreement terms at the URLs listed above.
Are you sure you would like to continue? [y/N] ^C
```
